In [1]:
%%sh
lspci | grep -i nvidia
echo "---"
cat /proc/driver/nvidia/version
echo "---"
nvidia-smi

00:1e.0 3D controller: NVIDIA Corporation GK210GL [Tesla K80] (rev a1)
---
NVRM version: NVIDIA UNIX x86_64 Kernel Module  418.67  Sat Apr  6 03:07:24 CDT 2019
GCC version:  gcc version 7.2.1 20170915 (Red Hat 7.2.1-2) (GCC) 
---
Tue Aug 13 13:54:26 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   50C    P0    79W / 149W |      0MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                  

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11890442876711808975
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14292989420657240792
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18091739136589818926
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8219389199979211436
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.callbacks import TensorBoard
print(tf.__version__)

1.14.0


In [4]:
# データインポート
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
x_train = x_train.reshape(60000, 784)
x_train = x_train/255.
x_test = x_test.reshape(10000, 784)
x_test = x_test/255.

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [8]:
# ネットワークの構築
model = Sequential()
# 中間層
model.add(
    Dense(
        units=64,
        input_shape=(784,),
        activation="relu"
    )
)
model.add(
    Dense(
        units=10,
    activation="softmax"
    )
)

W0813 13:54:29.248596 140087214024512 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
tsb = TensorBoard(log_dir="../logs")
history_adam = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    callbacks=[tsb]
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 5s 102us/sample - loss: 0.3343 - acc: 0.9076 - val_loss: 0.1893 - val_acc: 0.9484
Epoch 2/20
48000/48000 [==============================] - 5s 97us/sample - loss: 0.1651 - acc: 0.9531 - val_loss: 0.1457 - val_acc: 0.9568
Epoch 3/20
48000/48000 [==============================] - 5s 100us/sample - loss: 0.1215 - acc: 0.9646 - val_loss: 0.1268 - val_acc: 0.9619
Epoch 4/20
48000/48000 [==============================] - 5s 97us/sample - loss: 0.0950 - acc: 0.9724 - val_loss: 0.1077 - val_acc: 0.9674
Epoch 5/20
48000/48000 [==============================] - 5s 97us/sample - loss: 0.0780 - acc: 0.9768 - val_loss: 0.1019 - val_acc: 0.9711
Epoch 6/20
48000/48000 [==============================] - 5s 97us/sample - loss: 0.0638 - acc: 0.9809 - val_loss: 0.1023 - val_acc: 0.9690
Epoch 7/20
48000/48000 [==============================] - 5s 98us/sample - loss: 0.0535 - acc: 0.9835 - val_loss: 

In [10]:
history_adam.history["loss"], history_adam.history["acc"], history_adam.history["val_loss"], history_adam.history["val_acc"]

([0.33433545850714047,
  0.16506498453207313,
  0.121485820222646,
  0.09503053862089292,
  0.07800709932499254,
  0.06375332250826371,
  0.0535361955803819,
  0.04587861341723086,
  0.03882519414209916,
  0.032278310815648484,
  0.0294405776373848,
  0.024078686788726676,
  0.021087151254701893,
  0.01799577595463294,
  0.017128750631265576,
  0.013733074542073155,
  0.012723390270492624,
  0.011433174278119016,
  0.01051610708683317,
  0.008417590318143387],
 [0.9076458,
  0.95314586,
  0.964625,
  0.97241664,
  0.97675,
  0.9808958,
  0.9835,
  0.9854583,
  0.9882917,
  0.9904375,
  0.991,
  0.9927292,
  0.994,
  0.99472916,
  0.99516666,
  0.99614584,
  0.9967292,
  0.9966875,
  0.99733335,
  0.998],
 [0.18930611083408197,
  0.14573001246154307,
  0.12679820483674606,
  0.1076954030158619,
  0.10187103798923393,
  0.10227234617372354,
  0.09912006964224081,
  0.10186158854948978,
  0.10032427547747891,
  0.10686030963684122,
  0.10211064234174168,
  0.11142747613655714,
  0.1125151